In [2]:
import os, sys
from copy import deepcopy

In [3]:
from libs import *

In [4]:
from libs.pitsianis_van_loan_rank_1 import *

In [5]:
######################################################

In [6]:
def resolve_ket_vector_into_components(ket_vector):
    """
    # Transforms a state vector into individual basis vectors
    # Example:
    # Consider a 4x1 state vector(column matrix) - Example below
    # >>> vector = array([[[1],
    #                      [0],
    #                      [1],
    #                      [0]]])
    #
    #
    # >>> resolve_ket_vector_into_components(vector)
    # returns
    # >>> [ array([[1],[0],[0],[0]]), array([[0],[0],[1],[0]])]
    #
    # In General Diract notation sense,
    # |00>  is resolved into [|00>, |10>] where (x) is Kronecker Product 
    # 
    # Input : column matrix of dimension nx1 where n is greater than 2
    # Type  : numpy.ndarray
    #
    # return : list of numpy arrays
    #                   
    """
    #####
    def inner(x, y):
        if isinstance(x, complex):
            x = (x*x.conjugate()).real
            x = np.sqrt(x)
        if isinstance(y, complex):
            y = (y*y.conjugate()).real
            y = np.sqrt(y)
            
        if x*y>0:
            return 1
        elif x*y<0:
            return -1
        else:
            return 0
    #####
    if not isinstance(ket_vector, numpy.ndarray):
        raise Exception("Not a valid ket vector")
    ncols, nrows = ket_vector.shape
    if not (ncols>1 and nrows==1):
        raise Exception("Not a well defined ket vector")


    t_ket_vector = ket_vector.T.tolist()
    I_n = eye(ncols)
    resolved_vector_array = list()
    for i_n in I_n:
        t_vector = list(map(inner, t_ket_vector[0], i_n))
        v_sum = sum(t_vector)
        if v_sum!=0:
            base_vec= array([t_vector]).T
            factor = matmul(base_vec.T, ket_vector)[0, 0]
            resolved_vector_array.append((factor.item(), array([t_vector]).T ))
   

    return resolved_vector_array

In [ ]:
def convert_to_dirac_ket(list_of_ket_vectors):
    """
    # Converts a list of basis vectors(column matrix of dim 2x1) into Dirac's Ket notation
    # Example
    # >>> convert_to_diract_ket([array([[[1],[0]]]), array([[[0],[1]]]), array([[[1],[0]]])])
    # >>> |0>|1>|0>
    # argument: List of arrays(numpy.ndarray) each of dimension 2x1
    # return : type <str>
    """
    condition1 = isinstance(list_of_ket_vectors, list)
    condition2 = [isinstance(ket_vector, numpy.ndarray) for ket_vector in list_of_ket_vectors]
    if not (condition1 and condition2):
        raise Exception("TBA")
    try:
        ket_vector_str_list = list()
        for ket_vector in list_of_ket_vectors:
            if allclose(ket_vector, KET_0):
                ket_vector_str_list.append("|0>")
                continue
        
            if allclose(ket_vector, KET_1):
                ket_vector_str_list.append("|1>")
                continue

            if allclose(ket_vector, KET_XP):
                ket_vector_str_list.append("|+>")
                continue
                
            if allclose(ket_vector, KET_XM):
                ket_vector_str_list.append("|->")
                continue

            if allclose(ket_vector, KET_YP):
                ket_vector_str_list.append("|i+>")
                continue
            
            if allclose(ket_vector, KET_YM):
                ket_vector_str_list.append("|i->")  
                continue
            

        return "".join(ket_vector_str_list)
                
                
    except:
        raise Exception("TBA")

In [7]:
def nearest_kronecker_product(Psi, endian="little"):
    Endianness = ["big", "little"]
    endian = endian.lower()
    ncols, nrows = Psi.shape
    RESULTANT_KET_VECTORS = list()
    if not endian in Endianness:
        raise Exception("Unknown endian type. big and little are only accepted")

    if ncols > 2 and nrows == 1:
        if endian=="little":
            m1, m2 = ncols//2, 1
            n1, n2=  2, 1

        if endian=="big":
            m1, m2 = 2, 1
            n1, n2=  ncols//2, 1


    mx1, nx1 = m1, n1
    state_vector_image = deepcopy(Psi)
    ##### While Loop #####
    while state_vector_image.shape[0]>1:
        M1, M2 = pitsianis_van_loan_rank_1(state_vector_image, mx1, 1, nx1, 1)
        #####--------------------#####
        if M1.shape[0]>2 and M2.shape[0]==2:
            if allclose(reduce(kron, [M1, M2]), state_vector_image):
                RESULTANT_KET_VECTORS.append(M2)
                state_vector_image = M1
                mx1 = mx1//2
            else:
                RESULTANT_KET_VECTORS.append(state_vector_image)
                break
                
        #####--------------------#####           
        if M1.shape[0]==2 and  M2.shape[0]==2:
            if allclose(reduce(kron, [M1, M2]), state_vector_image):
                RESULTANT_KET_VECTORS.extend([M1, M2])
                break

            else:
                RESULTANT_KET_VECTORS.append(state_vector_image)
                break
        #####--------------------#####
        if M1.shape[0]==2 and M2.shape[0]>2:
            if allclose(reduce(kron, [M1, M2]), state_vector_image):
                nx1=nx1//2
                RESULTANT_KET_VECTORS.append(M1)
                state_vector_image = M2
                continue


    return RESULTANT_KET_VECTORS